In [55]:
import numpy as np
from elasticsearch import helpers, Elasticsearch
import csv
import pandas as pd
import os
import time
from tqdm import tqdm
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from inflection import singularize
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine as cs

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.info()


{'name': 'm-c02wv1jnhtd5',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'fpFxjw6cRjyGQOMl6ThLdw',
 'version': {'number': '7.5.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'e9ccaed468e2fac2275a3761849cbee64b39519f',
  'build_date': '2019-11-26T01:06:52.518245Z',
  'build_snapshot': False,
  'lucene_version': '8.3.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [4]:
def text_preprocessing(text):
    text= text.lower()
    text= re.sub(r'[^a-z]',' ',text)
    #text= " ".join([s for s in text.split() if len(s)>2])
    #text= " ".join([x for x in text.split() if x not in stopwords.words('english')])
    #text= " ".join([inflection.singularize(x) for x in text.split()])
    #text= ' '.join(text.split())
    return(text)

In [7]:
file_path = '/Users/s0c02nj/Downloads/000000000000.csv'
df_text = pd.read_csv(file_path)

#Removing the Nulls
df_text = df_text[df_text['abstract'].notna()]

In [70]:
doc1 = df_text['abstract'].iloc[7704]

In [9]:
df_text.shape

(307855, 1)

In [11]:
df_text['abstract'] = df_text['abstract'].apply(lambda x: text_preprocessing(x))


#### The process of Indexing

In [12]:
index='index_abstract'

In [13]:
def doc_generator(data,index):
    df_iter= data.iterrows()
    
    for index,document in df_iter:
        yield{
            "_index": 'index_abstract',
            "_type": "_doc",
            "_id" : f"{index}",
            "_source": document.to_json()
        }
    raise StopIteration
    

In [14]:
for success,info in tqdm(helpers.parallel_bulk(es,doc_generator(df_text,index))):
    if not success:
        print('A document failed:', info)
es.indices.refresh(index=index)

307855it [01:22, 3737.77it/s]


{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [15]:
def SEARCH(text,index,field):
    
    res=es.search(index=index,body={"query":{"match":{field:{"query":text,"operator":"or","fuzziness": "0"
                                                            }}}},size = 5)
    
    return([(x.get('_source'),x.get('_score')) for x in res['hits']['hits']])



In [21]:
index='index_abstract'
field = 'abstract'

In [77]:
SEARCH('what is corona',index,field)

[({'abstract': 'as soon as they enter a biological medium  cell culture medium for in vitro   blood or plasma for in vivo studies   nanoparticles  in most cases  see their surface covered by biomolecules  especially proteins  what the cells see is thus not the ideal nanoparticle concocted by chemists  meaning the biomolecular corona could have great biological and physiological repercussions  sometimes masking the expected effects of purposely grafted molecules  in this review  we will mainly focus on gold nanoparticles  in the first part  we will discuss the fate of these particles once in a biological medium  especially in terms of size  and the protein composition of the corona  we will highlight the parameters influencing the quantity and the identity of the adsorbed proteins  in a second part  we will resume the main findings about the influence of a biomolecular corona on cellular uptake  toxicity  biodistribution and targeting ability  to be noticed is the need for standardized 

In [24]:
# def query_generator(df):
#     while True:
#         yield (df_query['question'])

In [25]:
#query_generator(df_query)

In [26]:
#import concurrent.futures

In [27]:
# start = time.time()
# queries=[]
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     queries={executor.submit(SEARCH, query,index,'abstract'): query for query in df_query['question']}
# print(time.time()-start)


In [28]:
#queries
#SEARCH('corn','std_ref','qarth_product_name')

In [29]:
#out = [x.result() for x in queries]